In [1]:
from efficientnet_pytorch import EfficientNet
import torch

In [2]:
def get_net():
    net = EfficientNet.from_name('efficientnet-b1')
    net._fc = torch.nn.Linear(in_features=1280, out_features=4, bias=True)
    return net

In [3]:
%%capture
net = get_net().cuda()
checkpoint = torch.load('final_b1.pt')
net.load_state_dict(checkpoint['model_state_dict'])
net.set_swish(memory_efficient=False) # necessary step for ONNX export
net.eval()

In [4]:
x = torch.randn(1, 3, 512, 512, requires_grad=True).cuda()
torch_out = net(x)
torch.onnx.export(net,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "final_b1.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable lenght axes
                                'output' : {0 : 'batch_size'}})